In [1]:
from Dataset_Loader import load_leafs_dataset
from Training import train_cnn
from torch import nn, optim, manual_seed, flatten
from Util import apply_conv, apply_pool

d:\Downloads\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
batch_size = 20
test_split_size = 0.2

images_train, images_test, labels_train, labels_test = load_leafs_dataset(
    test_split_size,
    batch_size 
)

Reading CSV: 0.323 seconds
Encoding Labels: 0.021 seconds
Resizing Images: 15.539 seconds
Splitting Data: 1.922 seconds
Converting To Tensor: 19.277 seconds
Dividing To Batches: 4.083 seconds


In [6]:
input_images = images_train[:]
output_labels = labels_train[:]

In [7]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        width = 1633
        height = 1089
        self.dropout = nn.Dropout(0.2)

        self.pool1, width, height = apply_pool(width, height, kernel_size=2, stride=2)
        self.pool2, width, height = apply_pool(width, height, kernel_size=2, stride=2)
        self.pool3, width, height = apply_pool(width, height, kernel_size=2, stride=2)
    
        self.conv1, width, height = apply_conv(width, height, 1, 6, kernel_size=3, stride=1, padding=0)
        self.pool4, width, height = apply_pool(width, height, kernel_size=2, stride=2)

        self.conv2, width, height = apply_conv(width, height, 6, 16, kernel_size=3, stride=1, padding=0)
        self.pool5, width, height = apply_pool(width, height, kernel_size=2, stride=2)

        self.fc1 = nn.Linear(16*width*height, 1000)
        self.fc2 = nn.Linear(1000, 99)

    def forward(self, x):
        x = self.pool1(x)
        x = self.pool2(x)
        x = self.pool3(x)

        x = nn.functional.relu(self.conv1(x))
        x = self.pool4(x)

        x = nn.functional.relu(self.conv2(x))
        x = self.pool5(x)

        x = flatten(x, 1)
        x = nn.functional.relu(self.fc1(x))
        # x = self.dropout(x)
        x = nn.functional.log_softmax(self.fc2(x), dim=1)
        # x = self.dropout(x)
        return x

In [8]:
manual_seed(1)
model = CNN()
trained_model, train_losses = train_cnn(
    model=model,
    input_tensor=input_images,
    output_tensor=output_labels,
    epochs=15,
    batch_size=batch_size,
    lossFunction=nn.CrossEntropyLoss(),
    optimizer=optim.Adam(model.parameters(), lr=0.01, weight_decay=0.1),
    print_loss=True    
)

Epoch: 0 / loss: 6.973670959472656
Epoch: 1 / loss: 7.638968467712402
Epoch: 2 / loss: 7.086106777191162
Epoch: 3 / loss: 6.45104455947876
Epoch: 4 / loss: 6.630302429199219
Epoch: 5 / loss: 6.785460472106934
Epoch: 6 / loss: 7.629644870758057
Epoch: 7 / loss: 6.116540908813477
Epoch: 8 / loss: 6.517845153808594
Epoch: 9 / loss: 6.269039154052734
Epoch: 10 / loss: 5.8372578620910645
Epoch: 11 / loss: 5.567553520202637
Epoch: 12 / loss: 6.417655944824219
Epoch: 13 / loss: 6.045745849609375
Epoch: 14 / loss: 6.50205135345459
